In [ ]:
#This is a hack to make the %%cpp work inside mybinder.org - ignore
from ROOT import TObject

In [ ]:
%%cpp
//We specify that we are writing C++, rather than python, with the above %%cpp
//Create a histogram with 100 bins and a range from -10 to 10 with name "h" and titile "My First Histogam"
//The name 'h' must be unique - ROOT has an internal map of histogram names to memory locations, so each
//name must map to one location.
TH1F hist("h","My First Histogram;x {0}; frequency",100,10,10);
//Fill it with 2000 random numbers drawn from a Gaussian shape
hist.FillRandom("gaus",2000);

In [ ]:
%%cpp
//Note when using ROOT from a linux terminal, it turns out you can skip the two lines about the TCanvas - 
//this happens automatically behind the scenes
TCanvas can;
hist.Draw();
can.Draw();

In [ ]:
#Now lets try the same in python
#In the python case we have to explicitly ask to import the classes (TH1F, TCanvas) that we will use.
#We did not have to do anything equivalent in the C++ example above, executed in CLING c++ - CLING C++ is aware 
#of the ROOT libraries by default, python is not aware by default (hence the need for import statements). 
from ROOT import TH1F, TCanvas
histPython = TH1F("hPython","My First Histogram;x {0}; frequency",100,10,10)
histPython.FillRandom("gaus",2000);
canPython = TCanvas()
histPython.Draw()
canPython.Draw()

In [ ]:
#Task 1 - fill a histogram with a landau function and draw it. Use either C++ or python.

In [ ]:
%%cpp
//Now lets look at unbinned data and graphs
//define arrays of floats x and y
float x[10];
float y[10];
//fill the arrays up with some data
for (int i = 0; i < 10; ++i) {x[i]=i+1;y[i]=x[i]*x[i];}

In [ ]:
%%cpp
//Now create a TGraph using the above data - we specify 10 bins (to match the array sizes of 10) and the arrays
//x and y to use for the data.
TGraph myGraph(10,x,y);
//Then create a TCanvas, cd into it and draw both the TGraph and the TCanvas
TCanvas canGraph;
canGraph.cd();
myGraph.Draw();
canGraph.Draw();

In [ ]:
#Now lets do the same in python - note the way we fill arrays is very different to c++!
from ROOT import TGraph
n=10
from array import array
x=array('f')
y=array('f')
for counter in range(n):
    x.append(counter+1)
    y.append(x[counter]*x[counter])

In [ ]:
myGraph = TGraph(n,x,y)
#Note we cd into our canvas sued for python, and empty it with the Clear() function - that is because we have already
#drawn on this canvas above. The new drawing won't appear without first doing that.
canPython.cd()
canPython.Clear()
myGraph.Draw()
canPython.Draw()

In [ ]:
#Fitting - we can fit any predefined shape to the graph
#Lets try a Gaussian shape ('gaus')
myGraph.Fit("gaus")
#Then we have to draw the graph again, followed by drawing the canvas again.
myGraph.Draw()
canPython.Draw()
#The c++ code would be very similar.
#How well does the fit reproduce the data?
#Note that in a ROOT session in a linux terminal, you could do the fit by right-clicking on your data curve
#and selecting appropriate options on the GUI. The GUI's are not supported in Jupyter unfortunately.

In [ ]:
#Task 2 - Can you work out which shape would fit the data best?
#Hint: Look in section 7.2.1 to find out about predefined fit function shapes:
#https://root.cern.ch/root/htmldoc/guides/users-guide/FittingHistograms.html

In [ ]:
#Example of how to fill a txt file with data and then create a TGraph from it
#The C++ equivalent, shown in the lecture notes, is not shown because C++ IO is not supported in Jupyter via %%cpp.
from ROOT import TMath
fout = open("testData.txt","w")
counter = TMath.Pi()
while counter <= TMath.TwoPi():
    value = (counter,TMath.Sin(counter))
    valueString = ' '.join(map(str,value))
    valueString += "\n"
    fout.write(valueString)
    counter+=TMath.Pi()/500
fout.close()
from ROOT import TGraph
myGraph = TGraph("testData.txt")
from ROOT import TCanvas
canPythonDataGraph = TCanvas()
canPythonDataGraph.cd()
myGraph.Draw()
canPythonDataGraph.Draw()

In [ ]:
#Task 4 - create a data file with a cosine function, then create a TGraph from that and plot it.
#Then also perform a fit to it.
#Hint - for the latter you would need a user defined fit function, as discussed in section 7.2.2.1
#https://root.cern.ch/root/htmldoc/guides/users-guide/FittingHistograms.html
#and use it in your call to Fit()

In [ ]:
%%cpp
//Load the file tree2.root via the TFile object
TFile f("tree2.root");
//print the contents of the file
f.ls();
//Note we can't use the TBrowser shown in the lecture notes inside a Jupyter notebook
//But we can still print properties of the tree t2
t2 = (TTree*)f.Get("t2;1");
t2->GetListOfBranches()->Print();
//t2->Scan("pid");

In [ ]:
#The same thing in python
from ROOT import TFile
myFile = TFile("tree2.root")
t2p = myFile.Get("t2;1")
t2p.GetListOfBranches().Print()

In [ ]:
%%cpp
TCanvas canTree;
canTree.cd();
//Draw 3D plot of vector components against each other, only for events with pid of 1
t2->Draw("vect[0]:vect[1]:vect[2]","1 == pid");
canTree.Draw();

In [ ]:
#Task 6 (Task 5 is in the lecture slides!)
#Draw the 3D plot of vect components with and without cuts applied, as shown in the lecture notes.
#Try out other choices of cuts, first scanning the variable you want to cut on in order to see possible values

In [ ]:
%%cpp
t2->Draw("destep>>hDEStep","destep");
h2 = static_cast<TH1F*>(gDirectory->Get("hDEStep"));
h2->GetMean()

In [ ]:
%%cpp
//Create an odd and even histogram for events with odd or even values of the pid variable
t2->Draw("destep>>hEven","!(pid%2)");
//Retrieve from the global directory, which is where the above command put hEven
hEven = static_cast<TH1F*>(gDirectory->Get("hEven"));
t2->Draw("destep>>hOdd","(pid%2)");
hOdd = static_cast<TH1F*>(gDirectory->Get("hOdd"));
//Scale both histograms to unity via a factor of 1 divided by their areas
hEven->Scale(1./hEven->Integral());
hOdd->Scale(1./hOdd->Integral());
//Make a copy of the even histogram
TH1F hDiff(*hEven);
//And then add the odd histogram to that with a weight of -1
hDiff.Add(hOdd,-1);
can.cd();
can.Clear();
hDiff.Draw();
can.Draw();

In [ ]:
#The same in python
t2p.Draw("destep>>hEven_TH1F","!(pid%2)")
#Import the global directory, which is where the above command put hEven_TH1F
from ROOT import gDirectory
#Retrieve from the global directory, which is where the above command put hEven
hEven_TH1F = gDirectory.Get("hEven_TH1F")
#Scale histogram to unity via a factor of 1 divided by its area
hEven_TH1F.Scale(1./hEven_TH1F.Integral())
#Repeat for odd case
t2p.Draw("destep>>hOdd_TH1F","(pid%2)")
hOdd_TH1F = gDirectory.Get("hOdd_TH1F")
#Make a copy of the even histogram
hDiff = TH1F(hEven_TH1F)
#And then add the odd histogram to that with a weight of -1
hDiff.Add(hOdd_TH1F,-1)
canPythonOddEven = TCanvas()
canPythonOddEven.cd()
hDiff.Draw()
canPythonOddEven.Draw()